In [409]:
# Imports
import os
import requests
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
from dateutil.relativedelta import *
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import  yfinance as yf
# To show a progress bar when doing lengthy operations
from tqdm.notebook import tqdm

In [410]:
# Get historical OHLCV data for target ticker
etf_df = yf.download(
    "STW.AX, VAS.AX, SPY.AX", 
    period="max"
)

[*********************100%***********************]  3 of 3 completed


In [411]:
# Review data to see the date ranges
display(etf_df.head())
display(etf_df.tail())
display(etf_df.shape)
display(etf_df.index)


Adj Close                    Close                     High  \
              SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX   
Date                                                                     
2008-01-02       NaN  61.169998    NaN    NaN  61.169998    NaN    NaN   
2008-01-03       NaN  60.590000    NaN    NaN  60.590000    NaN    NaN   
2008-01-04       NaN  60.740002    NaN    NaN  60.740002    NaN    NaN   
2008-01-07       NaN  59.349998    NaN    NaN  59.349998    NaN    NaN   
2008-01-08       NaN  59.020000    NaN    NaN  59.020000    NaN    NaN   

                                Low                     Open             \
               STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX   
Date                                                                      
2008-01-02  61.169998    NaN    NaN  61.169998    NaN    NaN  61.169998   
2008-01-03  60.590000    NaN    NaN  60.590000    NaN    NaN  60.590000   
2008-01-04  60.740002    NaN    NaN  60.740002    NaN    NaN  60.740002   
2008-01-07  59.349998    NaN    NaN  59.349998    NaN    NaN  59.349998   
2008-01-08  59.020000    NaN    NaN  59.020000    NaN    NaN  59.020000   

                  Volume                
           VAS.AX SPY.AX STW.AX VAS.AX  
Date                                    
2008-01-02    NaN    NaN      0    NaN  
2008-01-03    NaN    NaN      0    NaN  
2008-01-04    NaN    NaN      0    NaN  
2008-01-07    NaN    NaN      0    NaN  
2008-01-08    NaN    NaN      0    NaN

Adj Close                             Close             \
                SPY.AX     STW.AX     VAS.AX      SPY.AX     STW.AX   
Date                                                                  
2022-12-28  566.070007  64.389999  88.169998  566.070007  64.389999   
2022-12-29  559.520020  63.320000  87.440002  559.520020  63.320000   
2022-12-30  565.000000  63.509998  87.699997  565.000000  63.509998   
2023-01-03  561.349976  62.669998  85.820000  561.349976  62.669998   
2023-01-04  564.659973  63.500000  86.930000  564.659973  63.500000   

                             High                               Low  \
               VAS.AX      SPY.AX     STW.AX     VAS.AX      SPY.AX   
Date                                                                  
2022-12-28  88.169998  575.000000  64.800003  88.610001  566.070007   
2022-12-29  87.440002  560.609985  64.029999  87.820000  558.599976   
2022-12-30  87.699997  566.179993  63.840000  88.129997  565.000000   
2023-01-03  85.820000  567.020020  63.849998  87.650002  561.260010   
2023-01-04  86.930000  567.000000  63.560001  87.010002  564.659973   

                                        Open                       Volume  \
               STW.AX     VAS.AX      SPY.AX     STW.AX     VAS.AX SPY.AX   
Date                                                                        
2022-12-28  64.279999  87.879997  570.000000  64.750000  88.500000  562.0   
2022-12-29  63.119999  87.089996  560.609985  64.029999  87.820000  566.0   
2022-12-30  63.509998  87.699997  565.500000  63.720001  87.930000  152.0   
2023-01-03  62.299999  85.239998  567.020020  63.830002  87.650002  168.0   
2023-01-04  63.060001  86.309998  564.950012  63.080002  86.389999   28.0   

                              
            STW.AX    VAS.AX  
Date                          
2022-12-28   61408  759834.0  
2022-12-29  181484  280709.0  
2022-12-30   30139   96586.0  
2023-01-03  168559  310891.0  
2023-01-04   62284   54169.0

(3796, 18)

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2022-12-19', '2022-12-20', '2022-12-21', '2022-12-22',
               '2022-12-23', '2022-12-28', '2022-12-29', '2022-12-30',
               '2023-01-03', '2023-01-04'],
              dtype='datetime64[ns]', name='Date', length=3796, freq=None)

In [412]:
# Examine the column type and check for nulls
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3796 entries, 2008-01-02 to 2023-01-04
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   (Adj Close, SPY.AX)  2080 non-null   float64
 1   (Adj Close, STW.AX)  3796 non-null   float64
 2   (Adj Close, VAS.AX)  3459 non-null   float64
 3   (Close, SPY.AX)      2080 non-null   float64
 4   (Close, STW.AX)      3796 non-null   float64
 5   (Close, VAS.AX)      3459 non-null   float64
 6   (High, SPY.AX)       2080 non-null   float64
 7   (High, STW.AX)       3796 non-null   float64
 8   (High, VAS.AX)       3459 non-null   float64
 9   (Low, SPY.AX)        2080 non-null   float64
 10  (Low, STW.AX)        3796 non-null   float64
 11  (Low, VAS.AX)        3459 non-null   float64
 12  (Open, SPY.AX)       2080 non-null   float64
 13  (Open, STW.AX)       3796 non-null   float64
 14  (Open, VAS.AX)       3459 non-null   float64
 15  (Volume, SPY.AX)    

In [413]:
# Retain the key columns for each etf
etf_df = etf_df.drop(columns = ['Adj Close', 'High', 'Low', 'Open', 'Volume'])
etf_df.columns = etf_df.columns.droplevel()     # Changes the multilevel indexing on columns to single level


In [414]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2008-01-02,NaN,61.169998,NaN
2008-01-03,NaN,60.590000,NaN
2008-01-04,NaN,60.740002,NaN
2008-01-07,NaN,59.349998,NaN
2008-01-08,NaN,59.020000,NaN


In [415]:
#  Drop all nulls
etf_df = etf_df.dropna()
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2079 entries, 2014-10-13 to 2023-01-04
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SPY.AX  2079 non-null   float64
 1   STW.AX  2079 non-null   float64
 2   VAS.AX  2079 non-null   float64
dtypes: float64(3)
memory usage: 65.0 KB


In [416]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2014-10-13,216.729996,48.599998,65.360001
2014-10-14,213.949997,49.119999,66.059998
2014-10-15,217.089996,49.500000,66.480003
2014-10-16,212.434738,49.549999,66.599998
2014-10-17,213.059998,49.779999,66.839996


In [417]:
# Collect the top 10 stocks of the S&P 500 to use as proxy for the SPY.AX ETF
# Note - hardcoding the stocks as there does not appear to be an easier way to 
# programitcally retrieve the S&P500 constituent stocks
spy_top10 = [
    {
        'symbol': 'AAPL',
        'description': 'Apple'
    }, 
    {
        'symbol': 'MSFT',
        'description': 'Microsoft' 
    },
    {
        'symbol': 'AMZN',
        'description': 'Amazon' 
    },
    {
        'symbol': 'TSLA',
        'description': 'Tesla' 
    },
    {
        'symbol': 'GOOGL',
        'description': 'Google'  
    },
    {
        'symbol': 'GOOG', 
        'description': 'Google'
    },
    {
        'symbol': 'NVDA', 
        'description': 'Nvidia'
    },
    {
        'symbol': 'BRK.B', 
        'description': 'Berkshire'
    },
    {
        'symbol': 'META', 
        'description': 'Meta'
    },
    {
        'symbol': 'UNH',
        'description': 'United Health'
    }
]

In [418]:
# Retrieve alpaca credentials
load_dotenv()
alpaca_api_key = os.getenv('APCA-API-KEY-ID')
alpaca_secret_key = os.getenv('APCA-API-SECRET-KEY')
if not(alpaca_api_key) or not(alpaca_secret_key):
    print('Failed to load API credentials')

In [419]:
# Prepare to retrieve news from alpaca
alpaca_endpoint = 'https://data.alpaca.markets/v1beta1/news'

alpaca_headers = {
    'Apca-Api-Key-Id': alpaca_api_key,
    'Apca-Api-Secret-Key': alpaca_secret_key
}


In [420]:
# Get the news for each of spy's top 10 stocks
parameters = {
    'symbols': spy_top10[0]['symbol'],
    # 'start': pd.to_datetime(etf_df.index[-1] + relativedelta(months = -6)).strftime('%Y-%m-%d'),
    'start': pd.to_datetime(etf_df.index[0]).strftime('%Y-%m-%d'),
    'end': pd.to_datetime(etf_df.index[-1]).strftime('%Y-%m-%d'),
    'limit': 50,
    'include_content': False,
}

response = requests.get(
    url = alpaca_endpoint,
    headers = alpaca_headers,
    params = parameters
)

response.raise_for_status()

In [421]:
# Unpack the news from response
news_page = pd.DataFrame(response.json()['news'])


In [422]:
# Split the index into date and time
# news_page.index = pd.to_datetime(news_page['updated_at'])
# news_page.index = pd.MultiIndex.from_arrays([news_page.index.date, news_page.index.time], names=['date','time'])

In [423]:
# Create a news dataframe
# news_df = news_page.loc[news_page['headline'].str.contains(spy_top10[0]['description'], na=False, case=False)]
news_df = pd.DataFrame()
news_df = news_df.append(news_page, ignore_index=True)

In [424]:
# print(response.json()['next_page_token'])

In [425]:
# while(response.json()['next_page_token'] != None):
#     parameters['page_token'] = response.json()['next_page_token']
#     print(parameters)
#     response = requests.get(
#         url = alpaca_endpoint,
#         headers = alpaca_headers,
#         params = parameters
#     )

#     response.raise_for_status()
#     print(response.json()['next_page_token'])

In [426]:
# Loop through all the remaining news data

while(response.json()['next_page_token'] != None):
    parameters['page_token'] = response.json()['next_page_token']

    response = requests.get(
        url = alpaca_endpoint,
        headers = alpaca_headers,
        params = parameters
    )

    response.raise_for_status()

    news_page = pd.DataFrame(response.json()['news'])
    print(f"{news_page.iloc[-1]['headline']}")
    # news_page = news_page.loc[news_page['headline'].str.contains(spy_top10[0]['description'], na=False, case=False)]
    news_df = news_df.append(news_page, ignore_index=True)


Tesla Boosts Discounts, SEC Tightens Check On Auditors' Report On Crypto, Netflix Plans New Jersey Production Hub: Today's Top Stories
Sony Eyed $6B Investment In Japan's Smartphone Sensor Factory
Apple Shows Indecision Heading Into CPI Data, FOMC: Here's What To Watch
Microsoft Eyes Super App, Taking A Leaf Out Of Tencent's Playbook
Market Volatility Drops Further Following Upbeat Payrolls Report
As iPhone City Emerges Out Of Lockdown, Apple Analyst Mulls On Production And Cupertino's Future In China
6M Shortfall In Apple iPhone Pro Models - China's Foxconn's Plant Outrage Impact On Apple
Market Volatility Decreases Further After US Stocks Record Gains
Ten Stocks Trending on Discord for Thursday November 17, 2022: ARDX, DLO, NVDA, CTMX, INM, HUDI, BABA, AAPL, TGT, GLBL
This US Representative Just Bought Shares Of Airbnb And These 2 Dividend-Paying Stocks
Apple Settles AirPods' Lawsuit But Ongoing Legal Battle On Active Noise Cancellation Remains Sore Spot For iPhone Maker
If You Inves

In [427]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21503 entries, 0 to 21502
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   author      21503 non-null  object
 1   content     21503 non-null  object
 2   created_at  21503 non-null  object
 3   headline    21503 non-null  object
 4   id          21503 non-null  int64 
 5   images      21503 non-null  object
 6   source      21503 non-null  object
 7   summary     21503 non-null  object
 8   symbols     21503 non-null  object
 9   updated_at  21503 non-null  object
 10  url         21503 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.8+ MB


In [428]:
# Save data to csv as downloading from alpaca can take up to 10 minutes
news_df.to_csv(Path('./data/news.csv'))

In [292]:
news_df

,author,content,created_at,headline,id,images,source,summary,symbols,updated_at,url
0,Chris Katje,,2023-01-03T23:54:34Z,EXCLUSIVE: Top 10 Searched Tickers On Benzinga...,30267286,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Each trading day features hundreds of headline...,"[AAPL, AMAM, AMC, AMZN, ATNF, CEI, COSM, CVNA,...",2023-01-03T23:54:34Z,https://www.benzinga.com/general/biotech/23/01...
2,Benzinga Insights,,2023-01-03T19:00:16Z,What 15 Analyst Ratings Have To Say About Apple,30262627,[],benzinga,,[AAPL],2023-01-03T19:00:16Z,https://www.benzinga.com/analyst-ratings/23/01...
4,Adam Eckert,,2023-01-03T18:26:06Z,Why Apple Stock Is Falling Today,30261999,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Apple Inc (NASDAQ: AAPL) shares are making new...,[AAPL],2023-01-03T18:26:06Z,https://www.benzinga.com/trading-ideas/movers/...
9,Adam Eckert,,2023-01-03T16:01:56Z,Apple Tells Suppliers To Build Fewer Component...,30258995,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Apple Inc (NASDAQ: AAPL) shares slid into the ...,"[AAPL, FOSL, HNHPF, QCOM, SSNNF, SWKS, TSM]",2023-01-03T16:01:57Z,https://www.benzinga.com/news/23/01/30258995/a...
10,Benzinga Newsdesk,,2023-01-03T14:35:46Z,"Apple Said To Ask Suppliers For Fewer AirPods,...",30257892,[],benzinga,,[AAPL],2023-01-03T14:35:46Z,https://www.benzinga.com/news/23/01/30257892/a...
11,Anusuya Lahiri,,2023-01-03T14:28:44Z,Apple Hikes Battery Replacement Service Fee Fo...,30255601,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,\n\tApple Inc (NASDAQ: AAPL) raised service ch...,[AAPL],2023-01-03T14:28:44Z,https://www.benzinga.com/news/23/01/30255601/a...
13,Anusuya Lahiri,,2023-01-03T12:51:01Z,Shopify Could Emerge As Another Benefactor Of ...,30252710,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,\n\tShopify Inc (NYSE: SHOP) aimed to fill a p...,"[AAPL, AMZN, GOOG, GOOGL, META, SHOP]",2023-01-03T12:51:01Z,https://www.benzinga.com/news/23/01/30252710/s...
15,Benzinga Newsdesk,,2023-01-03T10:26:52Z,"Exane BNP Paribas Downgrades Apple to Neutral,...",30253010,[],benzinga,,[AAPL],2023-01-03T10:26:52Z,https://www.benzinga.com/news/23/01/30253010/e...
17,Michael Cohen,,2022-12-31T19:03:10Z,"Bulls And Bears Of The Week: Apple, AMC, Meta,...",30242858,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Benzinga examined the prospects for many inves...,"[AAPL, AMC, BTCUSD, LUV, META, MSTR, TSLA]",2023-01-01T02:10:43Z,https://www.benzinga.com/news/large-cap/22/12/...
23,Shanthi Rexaline,,2022-12-30T11:57:25Z,"$1,000 Invested In Apple Now Would Be Worth Th...",30234032,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,"Apple Inc. (NASDAQ: AAPL), often considered an...","[AAPL, HNHPF, TSLA]",2022-12-30T11:57:25Z,https://www.benzinga.com/news/22/12/30234032/1...


In [66]:
# Split the index into date and time
temp_df.index = pd.to_datetime(temp_df['updated_at'])
temp_df.index = pd.MultiIndex.from_arrays([temp_df.index.date, temp_df.index.time], names=['Date','Time'])
temp_df.head()

author content            created_at  \
Date       Time                                                        
2021-10-05 18:54:22        Randy Elias          2021-10-05T18:35:42Z   
           17:08:39  Benzinga Newsdesk          2021-10-05T17:08:39Z   
           15:57:08  Benzinga Newsdesk          2021-10-05T15:57:07Z   
           14:31:33        Adam Eckert          2021-10-05T14:31:33Z   
           08:45:36        Rachit Vats          2021-10-05T08:45:36Z   

                                                              headline  \
Date       Time                                                          
2021-10-05 18:54:22                        Why Apple Shares Are Rising   
           17:08:39  'Apple Pay lands in Bahrain with three major b...   
           15:57:08  Visa To Change Way It Processes Certain Apple ...   
           14:31:33  Is Now The Time To Buy The Dip In Apple And Mi...   
           08:45:36  Apple Taken To Court By Ericsson Over Allegati...   

                           id  \
Date       Time                 
2021-10-05 18:54:22  23247353   
           17:08:39  23245806   
           15:57:08  23244310   
           14:31:33  23240861   
           08:45:36  23233901   

                                                                images source  \
Date       Time                                                                 
2021-10-05 18:54:22  [{'size': 'large', 'url': 'https://cdn.benzing...          
           17:08:39                                                 []          
           15:57:08                                                 []          
           14:31:33  [{'size': 'large', 'url': 'https://cdn.benzing...          
           08:45:36  [{'size': 'large', 'url': 'https://cdn.benzing...          

                                                               summary  \
Date       Time                                                          
2021-10-05 18:54:22  Shares of companies in the broader technology ...   
           17:08:39                                                      
           15:57:08                                                      
           14:31:33  With the Nasdaq down more than 7.5% from its h...   
           08:45:36  Ericsson AB (NASDAQ: ERIC) on Monday filed a l...   

                          symbols            updated_at  \
Date       Time                                           
2021-10-05 18:54:22        [AAPL]  2021-10-05T18:54:22Z   
           17:08:39        [AAPL]  2021-10-05T17:08:39Z   
           15:57:08     [AAPL, V]  2021-10-05T15:57:08Z   
           14:31:33  [AAPL, MSFT]  2021-10-05T14:31:33Z   
           08:45:36  [AAPL, ERIC]  2021-10-05T08:45:36Z   

                                                                   url  
Date       Time                                                         
2021-10-05 18:54:22  https://www.benzinga.com/news/21/10/23247353/w...  
           17:08:39  https://www.benzinga.com/news/21/10/23245806/a...  
           15:57:08  https://www.benzinga.com/news/21/10/23244310/v...  
           14:31:33  https://www.benzinga.com/analyst-ratings/analy...  
           08:45:36  https://www.benzinga.com/news/21/10/23233901/a...

In [71]:
from transformers import pipeline


In [74]:
etf_pipeline = pipeline('sentiment-analysis')


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [91]:
# Transform a headline into a sentiment
def get_sentiment(headline):
    return etf_pipeline(headline)[0]

Why Apple Shares Are Rising
{'label': 'POSITIVE', 'score': 0.9917313456535339}


In [79]:
print(temp_df.iloc[4]['headline'])
etf_pipeline(temp_df.iloc[4]['headline'])

Apple Taken To Court By Ericsson Over Allegations Of Engaging In Unfair Practices In 5G Patent Royalty Talks


[{'label': 'NEGATIVE', 'score': 0.9977689981460571}]

In [81]:
print(temp_df.iloc[3]['headline'])
etf_pipeline(temp_df.iloc[3]['headline'])

Is Now The Time To Buy The Dip In Apple And Microsoft?


[{'label': 'NEGATIVE', 'score': 0.9965846538543701}]

In [82]:
print(temp_df.iloc[1]['headline'])
etf_pipeline(temp_df.iloc[1]['headline'])

'Apple Pay lands in Bahrain with three major banks supported at launch' -Apple Insider Report


[{'label': 'NEGATIVE', 'score': 0.8031097054481506}]

In [84]:
etf_pipeline('Apple Pay lands in Bahrain with three major banks supported at launch')

[{'label': 'POSITIVE', 'score': 0.9262937903404236}]

In [83]:
print(temp_df.iloc[2]['headline'])
etf_pipeline(temp_df.iloc[2]['headline'])

Visa To Change Way It Processes Certain Apple Pay Transactions, Change Will Trim Fees Banks Pay To Apple On Recurring Transactions


[{'label': 'NEGATIVE', 'score': 0.9922193288803101}]

In [86]:
print(temp_df.iloc[10]['headline'])
etf_pipeline(temp_df.iloc[10]['headline'])

Benzinga's Bulls And Bears Of The Week: Apple, Boeing, Moderna, Robinhood And More


[{'label': 'POSITIVE', 'score': 0.9982277750968933}]

In [17]:
stw_df = yf.Ticker('STW.AX')
stw_news = stw_df.news
len(stw_news)

8

In [20]:
stw_news[:8]

[{'uuid': '83d81499-1f54-322b-83f5-40cd7f4d0214',
  'title': 'Investors in Cliq Digital (ETR:CLIQ) have made a incredible return of 861% over the past three years',
  'publisher': 'Simply Wall St.',
  'link': 'https://finance.yahoo.com/news/investors-cliq-digital-etr-cliq-041349727.html',
  'providerPublishTime': 1672287229,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/gnGTQ_q9KCdl7Xvx_ppB0g--~B/aD00MzI7dz0xMTk0O2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 1194,
     'height': 432,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/ooX5AtoQCMx8g9kEgh.tsQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]}},
 {'uuid': 'd4095648-af2c-3f4c-8a38-3bc9a3830bfd',
  'title': 'NORMA Group SE (ETR:NOEJ) i

In [21]:
vas_df = yf.Ticker('VAS.AX')
vas_news = vas_df.news
vas_news

[{'uuid': '83d81499-1f54-322b-83f5-40cd7f4d0214',
  'title': 'Investors in Cliq Digital (ETR:CLIQ) have made a incredible return of 861% over the past three years',
  'publisher': 'Simply Wall St.',
  'link': 'https://finance.yahoo.com/news/investors-cliq-digital-etr-cliq-041349727.html',
  'providerPublishTime': 1672287229,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/gnGTQ_q9KCdl7Xvx_ppB0g--~B/aD00MzI7dz0xMTk0O2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 1194,
     'height': 432,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/ooX5AtoQCMx8g9kEgh.tsQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]}},
 {'uuid': 'd4095648-af2c-3f4c-8a38-3bc9a3830bfd',
  'title': 'NORMA Group SE (ETR:NOEJ) i

In [22]:
vas_news[-1]

{'uuid': '1e4bb96f-3899-3e11-a1bd-141924a4b16b',
 'title': 'Chinese Gaming Bonanza Ignites Tencent, Asian Gaming Rally',
 'publisher': 'Bloomberg',
 'link': 'https://finance.yahoo.com/news/chinese-gaming-bonanza-ignites-tencent-035410272.html',
 'providerPublishTime': 1672286050,
 'type': 'STORY',
 'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/FC4IjAMF3HDWsWZj_78d8A--~B/aD0xMzM0O3c9MjAwMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/bloomberg_technology_68/7862c3e6e7a393efc2d3f0aae966b154',
    'width': 2000,
    'height': 1334,
    'tag': 'original'},
   {'url': 'https://s.yimg.com/uu/api/res/1.2/ehfL7nJI1CqbinDYBYTjqA--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/bloomberg_technology_68/7862c3e6e7a393efc2d3f0aae966b154',
    'width': 140,
    'height': 140,
    'tag': '140x140'}]},
 'relatedTickers': ['TCEHY']}

In [23]:
tsla_df = yf.Ticker('TSLA')
tsla_news = tsla_df.news
tsla_news

[{'uuid': '7718706f-6809-36f4-b662-6a8d8e4c3205',
  'title': 'Dow Jones, Nasdaq Break Key Levels As Apple Skids, Tesla Ends Dive; What To Do Now',
  'publisher': "Investor's Business Daily",
  'link': 'https://finance.yahoo.com/m/7718706f-6809-36f4-b662-6a8d8e4c3205/dow-jones%2C-nasdaq-break-key.html',
  'providerPublishTime': 1672288907,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/mxVgBGHlXoOaKc5bmhSFUA--~B/aD01NjU7dz0xMDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/c858484a148a1a8104b2fdb9e68c2f4b',
     'width': 1000,
     'height': 565,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/yxTeuVj9n3HQjIEebHy46w--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/c858484a148a1a8104b2fdb9e68c2f4b',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['^DJI', 'COMP', '^GSPC', 'TSLA', 'AAPL', 'FSLR']},
 {'uuid': '85400be1-c6c8-324